In [92]:
import pandas as pd

In [95]:
df = pd.DataFrame({'col': [25, 26, 38, 34, 3, 33, 23, 85, 70, 28, 27, 3000]})
df

,col
0,25
1,26
2,38
3,34
4,3
5,33
6,23
7,85
8,70
9,28


In [3]:
df.describe()

,col
count,11.000000
mean,35.636364
std,22.795933
min,3.000000
25%,25.500000
50%,28.000000
75%,36.000000
max,85.000000


In [5]:
df.sort_values(by='col')

,col
4,3
6,23
0,25
1,26
10,27
9,28
5,33
3,34
2,38
8,70


In [13]:
df['col'].quantile(0.75)

36.0

In [20]:
import numpy as np

In [96]:
# Assuming a column follow a normal/gaussian distribution we will calculate the boundaries which differentiate the outliers
def determine_outlier_thresholds_std(dataframe, col_name):
    upper_boundary = dataframe[col].mean() + 3 * dataframe[col].std()
    lower_boundary = dataframe[col].mean() - 3 * dataframe[col].std()
    return lower_boundary, upper_boundary

def check_outliers_std(dataframe, col_name):
    lower_boundary, upper_boundary = determine_outlier_thresholds_zscore(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upper_boundary) | (dataframe[col_name] < lower_boundary)].any(axis=None):
        return True
    else: 
        return False

def replace_with_thresholds_std(dataframe, replace=False):
    from tabulate import tabulate
    data = []
    for col_name in dataframe.columns:
        if col_name != 'Outcome':
            outliers_ = check_outliers_std(dataframe, col_name)
            count = None
            lower_limit, upper_limit = determine_outlier_thresholds_std(dataframe, col_name)
            if outliers_:
                count = dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)][col].count()
                if replace:
                    if lower_limit < 0:
                        # We don't want to replace with negative values, right!
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
                    else:
                        dataframe.loc[(dataframe[col_name] < lower_limit), col_name] = lower_limit
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
            outliers_status = check_outliers_std(dataframe, col_name)
            data.append([outliers_, outliers_status,count, col_name, lower_limit, upper_limit])
    table = tabulate(data, headers=['Outlier (Previously)','Outliers','Count', 'Column','Lower Limit', 'Upper Limit'], tablefmt='rst', numalign='right')
    print("Removing Outliers using Z Score")
    print(table)

replace_with_thresholds_std(df, replace=False)

Removing Outliers using Z Score
======================  ==========  =======  ========  =============  =============
Outlier (Previously)    Outliers      Count  Column      Lower Limit    Upper Limit
======================  ==========  =======  ========  =============  =============
True                    True              1  col            -2285.38        2850.71
======================  ==========  =======  ========  =============  =============


In [88]:
lower_limit, upper_limit = determine_outlier_thresholds_std(df, 'col')
count = df[(df['col'] > upper_limit) | (df['col'] < lower_limit)]['col'].count()
count

0

In [89]:
check_outliers_std(df, 'col')

False